# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [30]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [31]:
stocks = pd.read_csv(r'C:\Users\user\Downloads\sp_500_stocks.csv')
type(stocks)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
499,YUM
500,ZBH
501,ZBRA
502,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [32]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [33]:
symbol='AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 90493146,
 'calculationPrice': 'close',
 'change': 4.02,
 'changePercent': 0.03583,
 'close': 119.15,
 'closeSource': 'fcifilao',
 'closeTime': 1707006475171,
 'companyName': 'Amazon.com Inc.',
 'currency': 'USD',
 'delayedPrice': 119.79,
 'delayedPriceTime': 1724186970416,
 'extendedChange': 0.098,
 'extendedChangePercent': 0.00085,
 'extendedPrice': 120.088,
 'extendedPriceTime': 1733646321672,
 'high': 120.8,
 'highSource': 'liaueedipr dm1e ney5 tc',
 'highTime': 1685540677789,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 117.4,
 'iexCloseTime': 1687179511074,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 116.66,
 'iexOpenTime': 1693260679312,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1722133540855,
 'latestPrice': 116.5,
 'latestSource': 'Close',
 'latestTime': 'June 24, 2022',
 'latestUpdate': 1672312030406,
 'latestVolume': 72936303,
 'low': 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [34]:
data['latestPrice']
data['marketCap']

1214653926932

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [35]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
sp_dataframe = pd.DataFrame(columns = my_columns)
sp_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [36]:
sp_dataframe = sp_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
sp_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,116.5,1214653926932,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [37]:
sp_dataframe = pd.DataFrame(columns = my_columns)

for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    print(api_url)
    data = requests.get(api_url).json()
    print(data)
    sp_dataframe = sp_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
    print(sp_dataframe)
    


https://sandbox.iexapis.com/stable/stock/A/quote?token=Tpk_059b97af715d417d9f49f50b51b1c448
{'avgTotalVolume': 1904641, 'calculationPrice': 'close', 'change': 2.71, 'changePercent': 0.02339, 'close': 122.56, 'closeSource': 'filiofca', 'closeTime': 1670258515387, 'companyName': 'Agilent Technologies Inc.', 'currency': 'USD', 'delayedPrice': 123.24, 'delayedPriceTime': 1679158387066, 'extendedChange': 0.11, 'extendedChangePercent': 0.00094, 'extendedPrice': 124.3, 'extendedPriceTime': 1701149976376, 'high': 125.91, 'highSource': 'pnyutd eer1c de iml5eia', 'highTime': 1694344980719, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 123.67, 'iexCloseTime': 1708026345998, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 125.26, 'iexOpenTime': 1731161387056, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1736562759053, 'latestPrice': 125.9, 'latestSource': 'Close', 'latestTime': 'June 24, 2022', '

{'avgTotalVolume': 1659512, 'calculationPrice': 'close', 'change': 3.89, 'changePercent': 0.0269, 'close': 148.68, 'closeSource': 'iloacfif', 'closeTime': 1657025486902, 'companyName': 'Amerisource Bergen Corp.', 'currency': 'USD', 'delayedPrice': 149.15, 'delayedPriceTime': 1732182474516, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 152.73, 'extendedPriceTime': 1733774618480, 'high': 152.04, 'highSource': 'm1uiiaddercteye e n5pl ', 'highTime': 1670421462196, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 154.63, 'iexCloseTime': 1724214409830, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 145.49, 'iexOpenTime': 1672666815902, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1679434359190, 'latestPrice': 149.35, 'latestSource': 'Close', 'latestTime': 'June 24, 2022', 'latestUpdate': 1707674059364, 'latestVolume': 4241457, 'low': 145.5, 'lowSource': '5ci  yiued1tpelnem

{'avgTotalVolume': 4021217, 'calculationPrice': 'close', 'change': 11.54, 'changePercent': 0.03019, 'close': 400.84, 'closeSource': 'ailoffic', 'closeTime': 1667493284336, 'companyName': 'Adobe Inc', 'currency': 'USD', 'delayedPrice': 401.5, 'delayedPriceTime': 1680191093057, 'extendedChange': -0.54, 'extendedChangePercent': -0.00138, 'extendedPrice': 399.5, 'extendedPriceTime': 1702982606752, 'high': 400.84, 'highSource': 'ncmi ui yedpe1dtr e5ela', 'highTime': 1658245113957, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 401.75, 'iexCloseTime': 1717495754177, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 381.62, 'iexOpenTime': 1691017556294, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1704288941086, 'latestPrice': 405.68, 'latestSource': 'Close', 'latestTime': 'June 24, 2022', 'latestUpdate': 1691174447188, 'latestVolume': 3332780, 'low': 382.44, 'lowSource': 'EeXprii Irce tam el',

KeyboardInterrupt: 

In [38]:
sp_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,125.90,37307630849,N/A
1,AAL,14.50,9367010630,N/A
2,AAP,184.60,11212688283,N/A
3,AAPL,147.92,2313821202817,N/A
4,ABBV,154.36,270200004220,N/A
5,ABC,149.35,31399819777,N/A
6,ABMD,268.06,11785579311,N/A
7,ABT,114.11,198784576543,N/A
8,ACN,302.75,204250014619,N/A
9,ADBE,405.68,190330323754,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

sp_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        sp_dataframe = sp_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
sp_dataframe

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [39]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:100000000


In [40]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,121.88,37161496644,6032
1,AAL,14.60,9341865466,50362
2,AAP,184.30,11569256070,3989
3,AAPL,146.74,2366636645133,5010
4,ABBV,156.86,273263066752,4687
...,...,...,...,...
131,DGX,144.25,17114099394,5097
132,DHI,68.14,23886239293,10790
133,DHR,258.10,190222776229,2848
134,DIS,98.35,179180278154,7476


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [41]:
writer = pd.ExcelWriter(r'C:\Users\user\Downloads\recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [42]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [22]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [43]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [44]:
writer.save()